In [ ]:
#Ноутбук по мотивам соревнования https://www.kaggle.com/competitions/news-scraping-competition
#Классификация новостей по темам:
#Здоровье/Забота о себе
#Происшествия/Силовые структуры
#Страны бывшего СССР
#Наука
#Спорт
#Туризм/Путешествия
#Общество
#Экономика
#Строительство/Недвижимость

In [ ]:
import pandas as pd
from collections import Counter
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, SequentialSampler)
from sklearn.model_selection import train_test_split

#Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#BERT
from transformers import BertTokenizer


import re
import numpy as np

import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=FutureWarning)

import random
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Preparation

In [ ]:
# lenta.ru
care = pd.read_csv('/kaggle/input/lentaru/care_lenta_2012-03-01_2023-12-22.csv') #5
ussr = pd.read_csv('/kaggle/input/lentaru/ex_ussr_lenta_2012-03-01_2023-12-22.csv') #3
police = pd.read_csv('/kaggle/input/lentaru/police_lenta_2012-03-01_2023-12-22.csv') #2
science = pd.read_csv('/kaggle/input/lentaru/science_lenta_2012-03-01_2023-12-22.csv')#8
sport = pd.read_csv('/kaggle/input/lentaru/sport_lenta_2012-03-01_2023-12-22.csv') #4
tourism = pd.read_csv('/kaggle/input/lentaru/tourism_lenta_2012-03-01_2023-12-22.csv') #7
society= pd.read_csv('/kaggle/input/lentaru/society_lenta_2012-03-01_2023-12-22.csv') #0
economy = pd.read_csv('/kaggle/input/lentaru/economy_lenta_2012-03-01_2023-12-22.csv') #1

In [ ]:
# ria.ru
realty_list = !ls /kaggle/input/ria-custom-topics/ria_realty
construction_list = !ls /kaggle/input/ria-custom-topics/ria_construction
health_list = !ls /kaggle/input/ria-custom-topics/ria_health
society_list= !ls /kaggle/input/ria-custom-topics/ria_society

cat_dict = {'realty':'ria_realty/', 'construction':'ria_construction/', 'health':'ria_health/', 'society' :'ria_society/'}

base = '/kaggle/input/ria-custom-topics/'

def clean_ria_source(somestring):
    pat = re.compile('([-—]\sРИА\sН(едвижимость|овости)\.)|(\/\sРадио\sSputnik\.)|(\n)')
    somestring = re.sub(pat, '', somestring)
    return somestring

def ria_set(some, cat):
    res = pd.DataFrame()
    for item in some:
        res = pd.concat([pd.read_csv(base+cat_dict[cat]+str(item)), res])
    res = res.rename(columns={'id': 'docid', 'content':'text'})
    res['text'] =  res['text'].apply(lambda x: clean_ria_source(x))
    return res

In [ ]:
realty = ria_set(realty_list, 'realty')
construction = ria_set(construction_list, 'construction')
health = ria_set(health_list, 'health')
ria_society = ria_set(society_list, 'society')

In [ ]:
#iz.ru
construction_list_iz = !ls /kaggle/input/iz-construction
base = '/kaggle/input/iz-construction/'
res = pd.DataFrame()
for item in construction_list_iz:
     res = pd.concat([pd.read_csv(base+str(item)), res])

construction_iz = res.rename(columns={'id': 'docid', 'content':'text'})
construction_iz['text'] =  construction_iz['text'].apply(lambda x: clean_ria_source(x))



In [ ]:
construction_iz.head()

,Unnamed: 0,docid,url,title,text,datetime
0,0,1506861,https://iz.ru/1506861/2023-05-02/bank-domrf-na...,Банк ДОМ.РФ назвал размер портфеля проектного ...,Портфель проектного финансирования жилищного с...,2023-05-02T12:03:19+03:00
1,1,1505827,https://iz.ru/1505827/2023-04-28/podriadchika-...,Подрядчика на строительство новой дороги опред...,"В Воронеже определили подрядчика, который займ...",2023-04-28T20:59:11+03:00
2,2,1505782,https://iz.ru/1505782/2023-04-28/bashennyi-kra...,Башенный кран упал на стройке школы в Кирове,"Башенный кран рухнул в Кирове в пятницу, 28 ап...",2023-04-28T19:28:28+03:00
3,3,1505781,https://iz.ru/1505781/2023-04-28/mer-voronezha...,Мэр Воронежа назначил нового главу управления ...,Руководитель Дирекции единого заказчика капита...,2023-04-28T19:23:03+03:00
4,4,1504929,https://iz.ru/1504929/2023-04-27/gosekspertiza...,Госэкспертиза одобрила проект расширения желез...,Главгосэкспертиза одобрила проект реконструкци...,2023-04-27T12:52:27+03:00


In [ ]:
construction_iz['text']

0      Портфель проектного финансирования жилищного с...
1      В Воронеже определили подрядчика, который займ...
2      Башенный кран рухнул в Кирове в пятницу, 28 ап...
3      Руководитель Дирекции единого заказчика капита...
4      Главгосэкспертиза одобрила проект реконструкци...
                             ...                        
480    Основной этап строительства третьего энергобло...
481    Губернатор Алтайского края Виктор Томенко попр...
482    В Татарстане на третьем этапе обхода Нижнекамс...
483    Путепровод в селе Дубки Симферопольского район...
484    На проспекте Ленина в Барнауле начался монтаж ...
Name: text, Length: 3086, dtype: object

In [ ]:
realty['text']

0      СОЧИ, 2 ноя  Самый большой фудхолл сети Eat Ma...
1      МОСКВА, 2 ноя  Президент России Владимир Путин...
2      МОСКВА, 2 ноя  Альфа-банк открыл Setl Group кр...
3      МОСКВА, 2 ноя  Краснодарская компания "Консалт...
4      МОСКВА, 2 ноя  X5 Group, управляющая сетями "П...
                             ...                        
972    СОЧИ, 2 ноя  Фудхоллы Eat Market могут появить...
973    МОСКВА, 2 ноя  Межрегиональное территориальное...
974    МОСКВА, 2 ноя  Президент России Владимир Путин...
975    СОЧИ, 2 ноя  Сеть турецких ресторанов Big Chef...
976    МОСКВА, 2 ноя  Вторичное жилье в крупных росси...
Name: text, Length: 6128, dtype: object

In [ ]:
realty = pd.concat([realty, construction, construction_iz], ignore_index=True)
society = pd.concat([society, ria_society], ignore_index=True)
care = pd.concat([health, care], ignore_index=True)

In [ ]:
care.loc[:, 'target'] = 5
ussr.loc[:, 'target'] = 3
police.loc[:, 'target'] = 2
science.loc[:, 'target'] = 8
sport.loc[:, 'target'] = 4
tourism.loc[:, 'target'] = 7
society.loc[:, 'target'] = 0
economy.loc[:, 'target'] = 1
realty.loc[:, 'target'] = 6


In [ ]:
realty.drop_duplicates(['text', 'title'], inplace=True)
care.drop_duplicates(['text', 'title'], inplace=True)
society.drop_duplicates(['text', 'title'], inplace=True)

In [ ]:
lens = []
for item in [care, ussr, police, science, sport, tourism, society, economy, realty]:
    lens.append(len(item))
lens

[14522, 70248, 53269, 52710, 65520, 28643, 109656, 73008, 12218]

In [ ]:
cols_to_keep = ['title']

In [ ]:
df = pd.concat([care, ussr, police, science, sport, tourism, society, economy, realty], ignore_index=True)
df = df[cols_to_keep]

In [ ]:
df.shape

(479794, 1)

In [ ]:
df.isna().any().sum()

0

In [ ]:
df.head(3)

,title
0,"В Роспотребнадзоре заявили, что штамм ""дельта""..."
1,"Названа приправа, у которой 300 полезных свойств"
2,Диетологи назвали неожиданные эффекты от ежедн...


In [ ]:
data_trainval, data_test = train_test_split(df, test_size=0.25, random_state = 112)
data_train, data_val = train_test_split(data_trainval, test_size=0.1, random_state = 112)

In [ ]:
data_train.to_csv('data_train.csv')
data_test.to_csv('data_test.csv')
data_val.to_csv('data_val.csv')


## Dataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
MAX_LEN = 2048
VOCAB_SIZE = 83828 #rubert-tiny2 vocab size
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')

In [ ]:
tokenizer.encode_plus('В Роспотребнадзоре заявили что')

{'input_ids': [2, 282, 74135, 31466, 1046, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

## Dataset

In [ ]:
#обучим свои эмбеддинги на токенизаторе rubert-tiny2
BATCH_SIZE=12

In [ ]:
MAX_SEQ_LEN = 30 # обрезаем длину заголовка
from torch.utils.data import Dataset

class CustomDataset(Dataset):

    def __init__(self, texts, tokenizer, max_len=MAX_SEQ_LEN):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]


        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=False,
            truncation=True,
            return_tensors='pt',
        )

        return {
           'text': text,
           'input_ids': encoding['input_ids'][:,:-1].flatten().to(device),

           'targets': encoding['input_ids'][:,1:].flatten().to(device),
            }



In [ ]:
[item for item in zip(tokenizer.all_special_tokens, tokenizer.all_special_ids)]


[('[UNK]', 1), ('[SEP]', 3), ('[PAD]', 0), ('[CLS]', 2), ('[MASK]', 4)]

## Classifier

In [ ]:
HIDDEN_DIM = 512
N_LAYERS = 2
BIDIRECTIONAL = False


In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self,  n_classes = VOCAB_SIZE, vocab_size = VOCAB_SIZE, embedding_dim=MAX_LEN, hidden_dim=HIDDEN_DIM, n_layers=N_LAYERS, bidirectional=BIDIRECTIONAL
     ):

        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(
            embedding_dim,
            hidden_dim,
            n_layers,
            bidirectional=bidirectional,
            dropout = 0.3,
            batch_first=True,
        )
        self.hidden_dim = hidden_dim
        self.output_dim = n_classes
        self.linear = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.projection = nn.Linear(self.hidden_dim, self.output_dim)
        self.func = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, inputs):

        inputs = self.embedding(inputs)

        outputs, (hidden, cell) = self.rnn(inputs)

       # outputs = torch.mean(outputs, dim=1)

        outputs = self.dropout(self.linear(self.func(outputs)))
        projection = self.projection(self.func(outputs))

        return projection


In [ ]:
train_dataset = CustomDataset(list(data_train['title']), tokenizer=tokenizer)
val_dataset = CustomDataset(list(data_val['title']), tokenizer=tokenizer)

    # create data loaders
train_data = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data =  DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
test_dataset = CustomDataset(list(data_test['title']),tokenizer=tokenizer)
test_data = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def generate_sequence(model, starter,  max_seq_len = MAX_SEQ_LEN) -> str:
    model = model.to('cpu')
    input_ids = tokenizer.encode_plus(starter)['input_ids']
    input_ids.pop(-1)
    sentence = torch.LongTensor(input_ids).to('cpu')
    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len - len(input_ids)):
            next_word_distribution = model(sentence)[-1]
            next_word_sample = torch.topk(next_word_distribution, 5)[1].detach().cpu().numpy()
            next_word = torch.LongTensor(np.random.choice(next_word_sample, 1)).to('cpu')
            sentence = torch.cat([sentence, next_word])

            if next_word.item() == 3:
                break

    sequence = tokenizer.decode(sentence, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return sequence

In [ ]:
def evaluate(model, criterion, dataloader) -> float:

    model.eval()
    perplexity = []

    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['targets'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

In [ ]:

def train(model, train_data, val_data, optimizer, criterion, epochs, model_name):

    losses = []
    train_perplexities = []
    val_perplexities = []
    best_perplexity = 10**9
    for epoch in range(epochs):
        model = model.to(device)
        epoch_losses = []
        model.train()
        for batch in train_data:

            optimizer.zero_grad()
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(logits, batch['targets'].flatten())
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

        current_train_loss = sum(epoch_losses) / len(epoch_losses)
        current_train_perplexity = evaluate(model, criterion, train_data)
        current_val_perplexity = evaluate(model, criterion, val_data)
        losses.append(current_train_loss)
        train_perplexities.append(current_train_perplexity)
        val_perplexities.append(current_val_perplexity)
        if current_val_perplexity < best_perplexity:
            torch.save(model, f'{model_name}_epoch_{epoch}.pt')
            best_perplexity = current_val_perplexity
        print(f'Epoch: {epoch}, loss: {current_train_loss}, train perplexity: {current_train_perplexity}, val perplexity: {current_val_perplexity}')
        print('Sample generation:', generate_sequence(model, 'В России'))
    return losses, train_perplexities, val_perplexities




In [ ]:
model = LSTMClassifier()
model = model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
#модель обучалась в несколько итераций совокупно 12 эпох
#Ниже сохранены метрики лучшей (предпоследней итерации) обучения.
#Для генерации будем использовать модель с перплексией 112,7 на валидационной выборке

In [ ]:
train(model4, train_data, val_data, optimizer, criterion, epochs=3, model_name='model4')

Epoch: 0, loss: 4.651966475332657, train perplexity: 57.660967679175506, val perplexity: 115.52200821853948

Sample generation: В России отреагировали против « нормандского формата »

Epoch: 1, loss: 4.440952158472476, train perplexity: 51.74927602596425, val perplexity: 112.73279510286896

Sample generation: В России назвали условия возобновления рейсов на курорты

Epoch: 2, loss: 4.389076871832019, train perplexity: 49.006671409793505, val perplexity: 113.29159223385436

Sample generation: В России разработали новый препарат лечения коронавируса и гриппа на COVID


([4.651966475332657, 4.440952158472476, 4.389076871832019],
 [57.660967679175506, 51.74927602596425, 49.006671409793505],
 [115.52200821853948, 112.73279510286896, 113.29159223385436])

In [ ]:
model = torch.load('/kaggle/input/model4epoch1/model4_epoch_1.pt')
criterion = nn.CrossEntropyLoss(ignore_index=0)


In [ ]:
#проверим качество модели на тестовой выборке
evaluate(model, criterion, test_data)

114.91209289807232

## Title Generation

In [ ]:
#Во время обучения мы выводили только одну последовательность, посмотрим, как работает генерация в целом

In [ ]:
for i in range(10):
    print(generate_sequence(model,'Стали известны'))


Стали известны личности погибших при пожаре во Внукском отеле в Абхазии детей с начала года на Урале на 2023 году до 2023 г
Стали известны условия возвращения в Европу
Стали известны детали нападения в России в ЕСПЧ на водителя
Стали известны подробности нападения россиян на инкассатор « Свидетели в доме на »
Стали известны личности погибших на территории России в Донбассе военных на фоне обстрела украинскими войсками ВСУ
Стали известны условия существования « АвтоВАЗ - Ареной нефти » после 2020 году
Стали известны детали задержания бывшего начальника Росры областей Украины в Минске и Петербурге за убийство отца в Минске
Стали известны подробности задержания подозреваемого во взяточничестве девочки из « банды » на Украине из России на восток России
Стали известны подробности задержания устроиавших бой в российской колонии
Стали известны личности убитых в российском офисе


In [ ]:
for i in range(10):
    print(generate_sequence(model,'Новые'))

Новые люди оказались под угрозой
Новые военные уничтожили « Градник на территории » из - под воды
Новые военные уничтожили украинский штурмовики на одном из подразделений ВСУ
Новые российские регионы получат новые учебные корабли на Украину через Керченского пролив
Новые люди в Москве устроили забастовку на месте преступления и устроили погромов на голове в Петербурге
Новые люди предложили сделать все российские города с помощью нейросетей для детей
Новые регионы предложили продлить запрет продажи рыбы
Новые российские города стали чаще жаловаться с коллегами
Новые российские компании оказались под угрозой дефолта
Новые школы оказались под ударом в Казахстане


In [ ]:
#Варианты почти не повторяются,  но многие предложения содержат грамматичекие ошибки, в чуть более длинных предложениях логическая связь заметно ослабевает
#Попробуем реализовать beam search
class Node:
    def __init__(self, sequence, prob = 1.0, idx = None):
        self.sequence = sequence
        self.prob = prob
        self.idx = idx
        self.childs = []

    def __repr__(self):
        return (
            f"Node\n sequence: {self.sequence.tolist()}, "
            + f"index: {self.idx}, prob: {self.prob}, "
            + f"num childs: {len(self.childs)}"
        )



def beam_search(
    model: torch.nn.Module, parents: list[Node], bins: int = 15, k: int = 8, eos_id: int = 3) -> torch.Tensor:
    """
    Generate sequence via beam search algorithm.

    Parameters
    ----------
    model : torch.nn.Module
        Trained pytorch model.
    parents : list[Node]
        List of parent nodes.
    bins : int, default = 8
        Maximum number of hypothesis.
    k : int, default = 2
        Number of searches.
    eos_id : int, default = 3
        End of sentence (<eos>) token id.

    Returns
    -------
    torch.Tensor of int's (1, N)
        Most probable hypothesis.
    """

    model.eval()
    model.to('cpu')
    childs = []
    for parent in parents:
        if parent.idx == eos_id:
            childs.append(parent)
        else:
            dist = model.forward(parent.sequence)[:, -1]
            probs, ids = torch.topk(torch.softmax(dist, dim=-1), k=k, dim=-1)
            probs /= torch.sum(probs)

            for prob, idx in zip(probs.flatten(), ids.flatten()):
                node = Node(
                    sequence=torch.cat((parent.sequence, idx.view(-1, 1)), 1),
                    idx=idx,
                    prob=parent.prob * prob,
                )
                # parent.childs.append(node) # uncomment to debug root node
                childs.append(node)

    if len(childs) > bins:
        childs, _ = zip(
            *sorted([(node, 1.0 - node.prob) for node in childs], key=lambda x: x[1])[:bins]
        )  # keep bins with highest prob

    # Terminate if <eos> appear in all sequences
    if all([node.idx == eos_id for node in childs]):
        priority = random.choice(np.argpartition([node.prob for node in childs], 10))
        return childs[priority].sequence

    return beam_search(model, childs, bins, k, eos_id)



In [ ]:

input_ids = tokenizer.encode_plus('В России')['input_ids']
input_ids.pop(-1)
ids = torch.LongTensor([input_ids]).to('cpu')


root = Node(ids)

for i in range(10):
    with torch.no_grad():
        seq = beam_search(model, [root]).flatten()

    print(tokenizer.decode(seq,skip_special_tokens=True, clean_up_tokenization_spaces=True))

В России за сутки госпитализировали 21 заболевших COVID - 19
В России оценили возможность снижения цен на нефть
В России назвали сроки возобновления авиасообщения с Россией
В России предложили ввести налог на тунеядство
В России отреагировали на заявление Кравчука о « зачистке »
В России заявили о готовности к отказу от поездки в Турцию
В России заявили о готовности к отказу от поездки в Европу
В России отреагировали на атаку ВСУ на Работино
В России отреагировали на слова Кравчука о « зачистке »
В России заявили о готовности к отказу от поездки в Турцию


In [ ]:
#предложения стали лаконичнее и корректнее, но есть повторы, при том, что алгоритм и так работает медленнее, чем хотелось бы
#видно, что повторяются в основном первые элементы последовательности.
#Объединим два генератора, чтобы добавить разнообразия. Пусть topk будет отвечать за голову, что обеспечит нам вариативность,
# а beam search за хвост, это не позволит предложению развалиться в конце, как часто бывает с topk

In [ ]:
def generate_title(model, starter, num):
    model.eval()
    model.to('cpu')
    with torch.no_grad():
        for i in range(num):
            starter_ext = generate_sequence(model, starter, max_seq_len = 4)
            input_ids = tokenizer.encode_plus(starter_ext)['input_ids']
            input_ids.pop(-1)
            ids = torch.LongTensor([input_ids]).to('cpu')

            root = Node(ids)
            seq = beam_search(model, [root]).flatten()
            print(tokenizer.decode(seq,skip_special_tokens=True, clean_up_tokenization_spaces=True))


In [ ]:
generate_title(model, 'Новые', 10)

Новые школы оказались под угрозой
Новые регионы Украины предложили переименовать страну в России
Новые регионы предложили продлить кредитные каникулы в России
Новые российские туристы оказались любителями картами
Новые регионы предложили повысить налоги
Новые люди устроили массовую драку на борту упавшего в море самолета
Новые школы научились зарабатывать на коронавирус
Новые военные учения оказались под угрозой
Новые военные учения получат « летающий радар »
Новые регионы в России предложили повысить налоги


In [ ]:
generate_title(model, 'В России', 10)

В России отреагировали на сообщения об отставке главы МИД Украины
В России оценили возможность дефицита топлива
В России назвали сроки возобновления полетов в Турцию
В России отреагировали на заявление Кравчука о « зачистке » в Донбассе
В России захотели наказывать за продажу алкоголя
В России захотели наказывать за продажу алкоголя из - за коронавируса
В России предложили ввести отдельный налог на бездетность
В России захотели наказывать за нарушение режима прекращения огня
В России захотели упростить продажу алкоголя
В России предложили увеличить расходы на оборону


In [ ]:
generate_title(model, 'В Москве', 10)

В Москве за сутки госпитализировали 23 пациента с COVID - 19
В Москве задержали участников оппозиционной акции в поддержку Pussy Riot
В Москве за сутки госпитализировали 190 пациентов с коронавирусом
В Москве прошел форум по борьбе с безработицей
В Москве мужчина расстрелял жену и покончил с собой
В Москве прошел форум " Россия "
В Москве за сутки госпитализировали 53 пациента с COVID - 19
В Москве мужчина выстрелил в грудь и попал под следствие
В Москве мужчина захватил заложников из магазина
В Москве мужчина открыл стрельбу по полицейским
